In [17]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [18]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"

    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "round summary" in dict.keys():
        # add round summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"].append(f"ROUND {current_round}: {dict["round summary"]}")
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    
    
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_cn_lot36.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


应经理的邀请，你来到了一家老旧的自助式仓库，由于仓库主人威尔逊去世，他正在拍卖一间仓库。这不是你第一次参加这种形式的拍卖会，就像一盒巧克力一样，没有人知道冰冷的卷帘门后面隐藏着潜在的宝藏。当你和其他感兴趣的买家围在拍卖师身边时，一名工人拿着一把断线钳将生锈的门锁拧开，一个堆满各种旧家具和古董的自助式仓库展现在眼前。拍卖师没有给竞拍者任何机会看清楚，就以 50 美元开始竞拍。

您的出价是多少？

possible actions:
1. $100.
2. $150.
3. $200.


I select option 2. TURN: 1 


你大声地喊出 "$150"，其他竞拍者看了看你，然后有人跳到了 $200。气氛变得紧张，每个人都在等待下一步。

possible actions:
1. $250.
2. $300.
3. $350.

I select option 2. TURN: 2 


你毫不犹豫地喊出 "$300"，这次没有人立即反应。拍卖师环视四周，寻找着下一个出价者。几秒钟的沉默后，另一名竞拍者迟疑地喊出了 "$350"。

possible actions:
1. $400.
2. $450（这是你口袋里所有的钱）。
3. 放弃，不再出价。

I select option 1. TURN: 3 


你果断地喊出 "$400"，这一次，四周再无声音。拍卖师重复你的出价，等待着可能不存在的更高出价。几秒后，他宣布：“一次...两次...三次，售出！”你赢得了这次竞标，尽管你并不完全确定里面藏着什么。

round summary: Kyle Jackson 在紧张的竞拍中以 $400 赢得了一间充满未知的自助式仓库。


前情提要:
"""
- ROUND 0: Kyle Jackson 在紧张的竞拍中以 $400 赢得了一间充满未知的自助式仓库。
"""


你跟随经理穿过无数的走廊，来到他光线昏暗的办公室，这里的家具和设备自上世纪 90 年代以来就没有改变过。他经常提醒你，这里的设施其实更加古老，从 60 年代就已经存在了。你迅速给你的债主Alex打了一个电话，并向他解释说，一旦你拿到钱，过几天就会还给他，而他的回答却是令人厌恶的威胁。经理拿出一个旧铁皮盒子放在他的办公桌上，对你伸出手，示意你付拍卖的钱，你答应了。在递给你自助式仓库的钥匙后，他催促你看一些旧的监控录像。

通过监控录像，他向你解释说，从他记事起，前房主每天都会来仓库。更奇怪的是，他在进入仓库之前会朝仓库跳三下，而且每次都会带一个纸袋，离开时纸袋总是空的。

不管这些奇怪的话和监控录像，你还是去停车场拿了你的行李袋，以便开始清理仓库，寻找有价值的东西卖掉。当你往回走时，一位中年妇女迎面向你走来，她请求你让她进入她的仓库（你上次买下了她的仓库），这样她就可以找回她祖传的家族银吊坠。

你下一步该怎么做？

possible actions:
1. 拒绝她
2. 接受她的请求，但失去潜在客户
3. 完全忽视她


...

I select option 1. TURN: 4 


你决定同意帮助达格教授进行仪式。尽管你内心充满了疑虑和恐惧，但你也明白这可能是解开仓库之谜的唯一机会。达格教授看到你的决定后，眼中闪过一丝欣慰的光芒。

他迅速地从他的包里拿出一些奇怪的物品，包括蜡烛、奇异的符号图纸、以及一些干枯的草药。他开始在房间中心的地板上布置这些物品，同时口中念念有词，似乎是在进行某种古老的咒语。

达格教授指示你站在一个特定的位置上，并告诉你在仪式进行时需要做的事情。他强调，无论发生什么，都不要离开你的位置，也不要打断仪式。随后，他点燃了蜡烛，整个房间被一种微弱而神秘的光芒所照亮。

仪式开始了，达格教授的声音变得更加响亮，空气中弥漫着一种奇怪的能量。突然间，房间震动起来，你感到一股强大的力量在聚集。这时，你注意到讲台上的书籍开始发光，光芒越来越亮，直到整个房间都被光芒所充满。

然而，就在这时，仪式被突如其来的尖叫声打断。你转头一看，只见那具枯萎的尸体开始动了，它的眼睛突然睁开，向你们投来了充满怨恨的目光。达格教授大声告诉你，仪式被打断了，现在你们必须尽快离开这里。

你的下一步行动是什么？

possible actions:
1. 尽可能快地跟着达格教授逃离房间。
2. 尝试对抗那具活动的尸体。
3. 寻找其他逃生的路线。

I select option 2. TURN: 5 


你决定尝试对抗那具活动的尸体，尽管达格教授的警告仍在耳边回响。你认为如果不解决这个立即的威胁，你们两个人都可能无法安全离开这个房间。你迅速地四处寻找可以用作武器的东西，最终你的目光落在了房间一角的一根沉重的铁杆上。

你拿起铁杆，稳固地握在手中，准备迎战那具枯萎的尸体。它以一种不自然的速度向你爬来，其动作迅捷而且充满了恶意。你可以看到它的眼睛里闪烁着一种邪恶的光芒，而它那空洞的面孔让人不寒而栗。

根据规则，尝试对抗这具枯萎的尸体需要进行一个技能检定，difficulty level medium。

skill: fighting
difficulty: medium

I am making a skill check using FIGHTING against a difficulty of MEDIUM.
And I rolled a 51 for a result of FAILURE. TURN: 6 


你紧握着铁杆，尽力向那具枯萎的尸体挥去，但在紧张和恐惧的影响下，你的动作变得笨拙和迟缓。尸体以惊人的速度躲开了你的攻击，然后迅速地向你扑来，将你推倒在地。你感到一股冰冷的触感覆盖了你的身体，而那具尸体的眼睛就在你的面前，充满了对生命的渴望和对死亡的恶意。

达格教授见状，立即冲过来试图拉开那具尸体，但尸体的力量远超他的预期。在这种情况下，你意识到对抗这具尸体可能不是明智的选择，必须尽快找到其他的逃生方法。

在这紧急的时刻，达格教授大声呼喊你，告诉你跟着他，他似乎找到了一条可能的逃生路线。你没有时间考虑，必须立即做出决定。

你的下一步行动是什么？

possible actions:
1. 尽量挣脱尸体的控制，跟着达格教授逃跑。
2. 继续尝试用铁杆对抗尸体，希望能找到反击的机会。
3. 呼喊求助，希望能有什么力量来救你。

I select option 1. TURN: 7 


你尽自己最大的努力挣扎，试图从那具枯萎尸体的控制中挣脱出来。恐惧和求生的本能让你的动作充满了绝望的力量。就在这时，达格教授也不顾一切地帮助你推开那具尸体，他的行动给了你逃脱的机会。

你终于挣脱了尸体的控制，迅速地站了起来，跟着达格教授向房间的另一侧跑去。尸体在你们身后怒吼，但你们没有回头，只是不断地向前冲。

达格教授带你来到了一个看似被遗忘的旧门前，他迅速地打开门，里面是一条狭窄的秘密通道。你们两个人匆忙进入，达格教授在进入后立即关闭了门，希望能暂时阻挡那具尸体的追击。

秘密通道昏暗而湿润，但你们没有时间去注意这些细节，只能尽可能快地向前移动。通道的尽头有一扇门，达格教授推开门，终于，你们看到了通往自由的出口。

你们冲出了仓库，深深地呼吸着新鲜空气。回头望去，仓库在夜色中显得格外阴森可怖，但至少现在，你们是安全的。

达格教授看着你，眼中充满了感激和敬意。他告诉你，今晚发生的一切都将成为他们研究的宝贵资料，但他也警告你，今后要远离这种黑暗的力量。他承诺会继续研究，试图找到封印那种邪恶存在的方法，同时也会确保你的安全。

round summary: 经历了一场惊心动魄的冒险后，Kyle Jackson和达格教授成功逃离了被诅咒的仓库。他们面对了无法想象的恐怖，但最终还是侥幸生还。这次经历让Kyle对超自然的世界有了更深的认识，也让他明白了某些东西是人类不应触碰的。尽管仓库的秘密仍然隐藏在黑暗中，但Kyle知道，有些谜题最好永远不要解开。

IndexError: list index out of range